In [31]:
import requests
from bs4 import BeautifulSoup as bs
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Initialize Firebase Admin

#cred = credentials.Certificate('movie-app-55963-firebase-adminsdk-tw1pv-68944fef40.json')
#firebase_admin.initialize_app(cred)

# Initialize Firestore
db = firestore.client()

# Define the Firestore document reference
doc_ref = db.collection("movie").document("d_link")

# Define URLs for each year
all_movie_categories = {
    "2024": "https://2moviesda.com/tamil-2024-movies/",
    "2023": "https://2moviesda.com/tamil-2023-movies/",
    "2022": "https://2moviesda.com/tamil-2022-movies/"
}

# Prepare to store data
concatenate = []

for year, url in all_movie_categories.items():
    # Initialize lists for each year
    image = []
    movieslist = []
    movie_poster_page = []
    movie_720 = []
    movie_server2 = []
    movie_server1 = []
    movie_pink = []
    movie_final = []

    # Send a GET request to the webpage
    req = requests.get(url)
    soup = bs(req.content, "html.parser")

    # Iterate through all <a> tags to find links
    for link in soup.find_all("a"):
        text = link.text.strip()
        if year in text:
            href = link.get("href")
            a = ("https://2moviesda.com" + href)
            movieslist.append(a)

    print(f"Movies list for {year}: {movieslist}")

    for i in movieslist:
        # Send a GET request to the movie page
        try:
            response = requests.get(i)
            response.raise_for_status()  # Ensure we notice bad responses
            soup = bs(response.content, "html.parser")

            # Extract image URLs
            img_tags = soup.find_all("img")
            image_urls = [img['src'] for img in img_tags if 'src' in img.attrs]
            for url in image_urls:
                image_links = ("https://2moviesda.com" + url)
                if "poster" in image_links:
                    image.append(image_links)

            # Navigate to main poster page
            for link in soup.find_all("a"):
                href = link.get("href")
                if href and ("download" in href or "/movie/page" in href):
                    a = ("https://2moviesda.com" + href)
                    movie_poster_page.append(a)

            # Extract 720p links
            for link in movie_poster_page:
                req = requests.get(link)
                soup = bs(req.content, "html.parser")
                for link in soup.find_all("a"):
                    text = link.text.strip()
                    if "720" in text:
                        href = link.get("href")
                        a = ("https://2moviesda.com" + href)
                movie_720.append(a)

            # Extract Server2 download links
            for link in movie_720:
                req = requests.get(link)
                soup = bs(req.content, "html.parser")
                for link in soup.find_all("a"):
                    href = link.get("href")
                    if href and ("download" in href or "movie" in href):
                        a = ("https://2moviesda.com" + href)
                movie_server2.append(a)

            # Extract Server1 download links
            for link in movie_server2:
                req = requests.get(link)
                soup = bs(req.content, "html.parser")
                for link in soup.find_all("a"):
                    href = link.get("href")
                    if href and ("download" in href or "movie" in href):
                        a = href
                movie_server1.append(a)

            # Extract final download links
            for link in movie_server1:
                req = requests.get(link)
                soup = bs(req.content, "html.parser")
                for link in soup.find_all("a"):
                    href = link.get("href")
                    if href and "720" in href:
                        a = href
                movie_final.append(a)

        except requests.RequestException as e:
            print(f"Error fetching the movie page: {e}")

    # Append the final links to the concatenate list
    concatenate.extend(movie_final)

# Remove '#' from the final list
final_download_links_filtered = [item for item in concatenate if item != '#']

# Data to be stored
data = {
    'image_url': image,
    'download_url': final_download_links_filtered,
}

# Store data in Firestore
doc_ref.set(data)
print(final_download_links_filtered)
print("Data stored successfully")


Movies list for 2024: ['https://2moviesda.com/pagutharivalan-2024-movie-download/', 'https://2moviesda.com/pogumidam-vegu-thooramillai-2024-movie-download/', 'https://2moviesda.com/vaazhai-2024-movie-download/', 'https://2moviesda.com/kottukkaali-2024-movie-download/', 'https://2moviesda.com/raayan-2024-hd-movie-download/', 'https://2moviesda.com/jama-2024-movie-download/', 'https://2moviesda.com/kalki-2898-ad-2024-movie-download-2/', 'https://2moviesda.com/murattu-singathin-adithadi-2024-movie-download/', 'https://2moviesda.com/idaivelai-2024-movie-download/', 'https://2moviesda.com/grrr-2024-movie-download/', 'https://2moviesda.com/raghu-thatha-2024-movie-download/', 'https://2moviesda.com/my-perfect-husband-2024-movie-download/', 'https://2moviesda.com/vedaa-2024-movie-download/', 'https://2moviesda.com/thangalaan-2024-movie-download/', 'https://2moviesda.com/demonte-colony-2-2024-movie-download/', 'https://2moviesda.com/konjam-pesinaal-yenna-2024-movie-download/']


KeyboardInterrupt: 